# Hierarchical Louvain

In [68]:
import math
import networkx as nx
import igraph as ig
import leidenalg as la
from ipysigma import Sigma, SigmaGrid
from ebbe import partitioned_items
from pelote import read_graphology_json

Sigma.set_defaults(max_categorical_colors=25)

In [63]:
corpus = read_graphology_json('eurosis.json')

# for node, attr in corpus.nodes.data():
#     del attr['x']
#     del attr['y']
#     del attr['color']

## Top-down approach

In [61]:
def leiden_modularity(g: nx.Graph, weighted: bool = False):
    ig_g = ig.Graph.from_networkx(g)

    weights = None

    if weighted:
        weights = [w for _, _, w in g.edges(data="weight")]
    
    partition = la.find_partition(ig_g, la.ModularityVertexPartition, weights=weights)
    return {n: m for n, m in zip(g, partition._membership)}

In [62]:
def hierarchical_leiden_modularity(g: nx.Graph):
    first_level = leiden_modularity(g)
    clusters = partitioned_items((v, k) for k, v in first_level.items())

    threshold = math.ceil(math.sqrt(g.order())) * 2

    result = {}
    
    for i, cluster in enumerate(clusters):
        if len(cluster) < threshold:
            for node in cluster:
                result[node] = str(i)

            continue
        
        subgraph = g.subgraph(cluster)
        second_level = leiden_modularity(subgraph)

        for j, sub_cluster in enumerate(partitioned_items((v, k) for k, v in second_level.items())):
            if j < 2:
                for node in sub_cluster:
                    result[node] = str(i) + '_' + str(j)
            else:
                for node in sub_cluster:
                    result[node] = str(i)

        # for node, j in second_level.items():
        #     result[node] = str(i) + '_' + str(j)

    return result

In [69]:
SigmaGrid(corpus, node_size=corpus.degree, views=[{"name": "Top level", "node_color": leiden_modularity(corpus)}, {"name": "Bottom level", "node_color": hierarchical_leiden_modularity(corpus)}])

## Bottom-up approach

In [55]:
partitions = list(nx.community.louvain_partitions(corpus, resolution=1))
len(partitions)

3

In [73]:
SigmaGrid(corpus, node_size=corpus.degree, views=[{"name": "Level " + str(i), "node_color": p} for i, p in enumerate(partitions)])

## Comparison

In [77]:
other = read_graphology_json('NETWORK_CorpusMedia_DEFACTO_medialab_SciencesPo_V1.json')

for node, attr in other.nodes.data():
    del attr['x']
    del attr['y']
    del attr['color']

other_partitions = list(nx.community.louvain_partitions(other, resolution=1))

In [79]:
len(other_partitions)

1

In [78]:
SigmaGrid(other, node_size=other.degree, views=[{"name": "Top-down", "node_color": other_partitions[0]}, {"name": "Bottom-up", "node_color": hierarchical_leiden_modularity(other)}])